# Подготовка данных <a id="chapter1"></a>

## Импорт библиотек <a id="chapter1_1"></a>

In [1]:
import pandas as pd
import time
import warnings
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer

def rmse(y, y_pred):
    res = mean_squared_error(y, y_pred)
    return res**0.5

## Просмотр общей информации <a id="chapter1_2"></a>

In [2]:
data = pd.read_csv('/datasets/autos.csv')

data.columns = data.columns.str.lower()

display(data.head(10))
display(data.info())
display(data.describe())

display(data['model'].value_counts())
display(data['vehicletype'].value_counts())
display(data['gearbox'].value_counts())
display(data['fueltype'].value_counts())
display(data['notrepaired'].value_counts())
display(data['brand'].value_counts())
display(data['power'].value_counts())

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
datecrawled          354369 non-null object
price                354369 non-null int64
vehicletype          316879 non-null object
registrationyear     354369 non-null int64
gearbox              334536 non-null object
power                354369 non-null int64
model                334664 non-null object
kilometer            354369 non-null int64
registrationmonth    354369 non-null int64
fueltype             321474 non-null object
brand                354369 non-null object
notrepaired          283215 non-null object
datecreated          354369 non-null object
numberofpictures     354369 non-null int64
postalcode           354369 non-null int64
lastseen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


None

,price,registrationyear,power,kilometer,registrationmonth,numberofpictures,postalcode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
serie_2                   8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: model, Length: 250, dtype: int64

...

manual    268251
auto       66285
Name: gearbox, dtype: int64

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: fueltype, dtype: int64

no     247161
yes     36054
Name: notrepaired, dtype: int64

volkswagen        77013
opel              39931
bmw               36914
mercedes_benz     32046
audi              29456
ford              25179
renault           17927
peugeot           10998
fiat               9643
seat               6907
mazda              5615
skoda              5500
smart              5246
citroen            5148
nissan             4941
toyota             4606
hyundai            3587
sonstige_autos     3374
volvo              3210
mini               3202
mitsubishi         3022
honda              2817
kia                2465
suzuki             2323
alfa_romeo         2314
chevrolet          1754
chrysler           1439
dacia               900
daihatsu            806
subaru              762
porsche             758
jeep                677
trabant             589
land_rover          545
daewoo              542
saab                526
jaguar              505
rover               486
lancia              471
lada                225
Name: brand, dtype: int64

0        40225
75       24023
60       15897
150      14590
101      13298
         ...  
16311        1
1360         1
1968         1
6226         1
6006         1
Name: power, Length: 712, dtype: int64

## Вывод <a id="chapter1_3"></a>

Имеем пропуски в пяти признаках: VehicleType, Gearbox, Model, FuelType, NotRepaired. 

1) Появилась идея заполнить пропуски в VehicleType, используя информацию из других записей, находя их по по схожим моделям. Например в 0-ой строке имеет пропуск в VehicleType, но при этом знаем модель: golf. Значит мы могли бы найти тип кузова в других записях с моделью golf, но проблема в том, что у одной и той же модели может быть большое количество типов кузова. Поэтому все пропуски в VehicleType заполняем значением other_1. 

2) Пропущенные значения в признаке Gearbox можно было бы также заполнить по предыдущей аналогии, но нет, так делать нельзя, т.к. автомобиль определенной модели и в определенном кузове может быть как с коробкой автомат, так и с механической коробкой переключения передач. Заполним аналогичным значением other_2. 

3) Таким же образом поступим с пропущенным значенями в признаке model. 

4) У прзнака fueltype есть дубликаты - это gasoline и petrol, сделаем замену на petrol. Пропуски в fueltype заполним other_4. 

5) Остается последний признак с большим количеством пропусков - NotRepaired. Пропуски могут быть обусловлены как и целенаправленным укрытием фактов дтп, так и их реальным отсутствием, поэтому пропуски заполним other_5. 

6) В признаке registrationyear присутствуют значения выше 2021, что не может быть правдой, поэтому отсавим только те строки, в которых год регистрации меньше, либо равен 2021. Также год регистрации не может быть меньше 1886.

7) В признаке price присутствуют автомобиле, стоимость которых равна 0, либо близка к 0. Введем допущение, что а/м не может стоить меньше 100 евро (в случае покупки а/м только ради металла и утилизации).

8) В признаке power оставим а/м, мощность которых меньше 2000 л.с. Почти у 30000 а/м в признаке power указано 0 л.с., поэтому заполним такие 0 средним значением.

9) В признаке numberofpictures везде 0, поэтому удалим данный столбец.

10) Скорее всего, дата скачивания анкеты из базы, дата создания анкеты и дата последней активности пользователя на стоимость автомобиля сильно не влияют, поэтому их мы тоже удалим.

11) В признаке registrationmonth присутствует нулевой месяц. Его так и оставим, понимая под этим значением категорию other.

## Предобработка данных <a id="chapter1_4"></a>

In [3]:
data['vehicletype'] = data['vehicletype'].fillna('other_1')
data['gearbox'] = data['gearbox'].fillna('other_2')
data['model'] = data['model'].fillna('otehr_3')
data['fueltype'] = data['fueltype'].str.replace("gasoline", "petrol")
data['fueltype'] = data['fueltype'].fillna('other_4')
data['notrepaired'] = data['notrepaired'].fillna('other_5')

data = data.query('1886 <= registrationyear <= 2021')
data = data.query('price >= 100')
data = data.query('power <= 2000')
data['power'] = data['power'].replace(0, round(data['power'].mean()))
data = data.drop(['numberofpictures', 'datecrawled', 'datecreated', 'lastseen'], axis = 1)

data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 340830 entries, 0 to 354368
Data columns (total 12 columns):
price                340830 non-null int64
vehicletype          340830 non-null object
registrationyear     340830 non-null int64
gearbox              340830 non-null object
power                340830 non-null int64
model                340830 non-null object
kilometer            340830 non-null int64
registrationmonth    340830 non-null int64
fueltype             340830 non-null object
brand                340830 non-null object
notrepaired          340830 non-null object
postalcode           340830 non-null int64
dtypes: int64(6), object(6)
memory usage: 33.8+ MB


# Обучение моделей <a id="chapter2"></a>

## Обучение LGBMRegressor <a id="chapter2_1"></a>

In [4]:
data_1 = data.copy()


data_1['vehicletype'] = data_1['vehicletype'].astype("category")
data_1['gearbox'] = data_1['gearbox'].astype("category")
data_1['model'] = data_1['model'].astype("category")
data_1['fueltype'] = data_1['fueltype'].astype("category")
data_1['brand'] = data_1['brand'].astype("category")
data_1['notrepaired'] = data_1['notrepaired'].astype("category")

features = data_1.drop(['price'], axis = 1)
target = data_1['price']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345) 

for i in range(10, 320, 50):
    for j in range(2, 11, 2):
        model = LGBMRegressor(max_depth = j, n_estimators = i)
        time_1 = time.time()
        model.fit(features_train, target_train)
        time_2 = time.time()
        prediction = model.predict(features_valid)
        time_3 = time.time()
        score = mean_squared_error(target_valid, prediction)
        print('Количество деревьев =', i, 'глубина дерева =', j, 'RMSE =', round(score**0.5), 'время на обучение =', round((time_2 - time_1), 1), 'время на предсказание =', round((time_3 - time_2), 1))         
        



Количество деревьев = 10 глубина дерева = 2 RMSE = 3228 время на обучение = 1.1 время на предсказание = 0.1
Количество деревьев = 10 глубина дерева = 4 RMSE = 2747 время на обучение = 1.4 время на предсказание = 0.1
Количество деревьев = 10 глубина дерева = 6 RMSE = 2593 время на обучение = 1.6 время на предсказание = 0.2
Количество деревьев = 10 глубина дерева = 8 RMSE = 2588 время на обучение = 1.7 время на предсказание = 0.2
Количество деревьев = 10 глубина дерева = 10 RMSE = 2588 время на обучение = 2.0 время на предсказание = 0.2
Количество деревьев = 60 глубина дерева = 2 RMSE = 2133 время на обучение = 2.7 время на предсказание = 0.3
Количество деревьев = 60 глубина дерева = 4 RMSE = 1828 время на обучение = 4.5 время на предсказание = 0.5
Количество деревьев = 60 глубина дерева = 6 RMSE = 1738 время на обучение = 5.9 время на предсказание = 0.7
Количество деревьев = 60 глубина дерева = 8 RMSE = 1730 время на обучение = 7.0 время на предсказание = 0.8
Количество деревьев = 60 гл

### Кросс-валидация для оценки качества и скорости работы модели LGBMRegressor <a id="chapter2_1_1"></a>

In [5]:
time_1 = time.time()
model = LGBMRegressor(max_depth = 10, n_estimators = 310)
score = cross_val_score(model, features, target, cv = 4, scoring = make_scorer(rmse))
time_2 = time.time()
print('Количество деревьев =', 310, 'глубина дерева =', 10, 'RMSE =', score.mean(), 'время на выполнение =', time_2 - time_1)        
        

Количество деревьев = 310 глубина дерева = 10 RMSE = 1620.3336596243612 время на выполнение = 114.99039387702942


## Обучение CatBoostRegressor <a id="chapter2_2"></a>

In [6]:
features = data.drop(['price'], axis = 1)
target = data['price']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345) 

category = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']

for i in range(10, 320, 50):
    for j in range(2, 11, 2):
        model = CatBoostRegressor(max_depth = j, n_estimators = i)
        time_1 = time.time()
        model.fit(features_train, target_train, verbose=False, cat_features = category)
        time_2 = time.time()
        prediction = model.predict(features_valid)
        time_3 = time.time()
        score = mean_squared_error(target_valid, prediction)
        print('Количество деревьев =', i, 'глубина дерева =', j, 'RMSE =', round(score**0.5), 'время на обучение =', round((time_2 - time_1), 1), 'время на предсказание =', round((time_3 - time_2), 1)) 



Количество деревьев = 10 глубина дерева = 2 RMSE = 4020 время на обучение = 4.5 время на предсказание = 0.2
Количество деревьев = 10 глубина дерева = 4 RMSE = 3871 время на обучение = 5.0 время на предсказание = 0.2
Количество деревьев = 10 глубина дерева = 6 RMSE = 3787 время на обучение = 6.5 время на предсказание = 0.2
Количество деревьев = 10 глубина дерева = 8 RMSE = 3749 время на обучение = 8.6 время на предсказание = 0.2
Количество деревьев = 10 глубина дерева = 10 RMSE = 3720 время на обучение = 8.0 время на предсказание = 0.2
Количество деревьев = 60 глубина дерева = 2 RMSE = 2920 время на обучение = 12.7 время на предсказание = 0.2
Количество деревьев = 60 глубина дерева = 4 RMSE = 2552 время на обучение = 21.0 время на предсказание = 0.2
Количество деревьев = 60 глубина дерева = 6 RMSE = 2365 время на обучение = 25.8 время на предсказание = 0.2
Количество деревьев = 60 глубина дерева = 8 RMSE = 2265 время на обучение = 32.3 время на предсказание = 0.2
Количество деревьев = 6

### Кросс-валидация для оценки качества и скорости работы модели CatBoostRegressor <a id="chapter2_2_1"></a>

In [7]:
time_1 = time.time()
model = CatBoostRegressor(max_depth = 10, n_estimators = 310, cat_features = category, verbose=False)
score = cross_val_score(model, features, target, cv = 4, scoring = make_scorer(rmse))
time_2 = time.time()
print('Количество деревьев =', 310, 'глубина дерева =', 10, 'RMSE =', score.mean(), 'время на выполнение =', time_2 - time_1)   

Количество деревьев = 310 глубина дерева = 10 RMSE = 1730.4373613921825 время на выполнение = 2126.7897572517395


## Обучение LinearRegression <a id="chapter2_3"></a>

In [8]:
warnings.filterwarnings('ignore')

data_2 = data.copy()
data_ohe = pd.get_dummies(data_2, drop_first=True)
display(data_ohe.head(10))


features = data_ohe.drop(['price'], axis = 1)
target = data_ohe['price']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345) 

numeric = ['registrationyear', 'power', 'kilometer', 'registrationmonth', 'postalcode']
scaler = StandardScaler()
scaler.fit(features_train[numeric]) 

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

model = LinearRegression()
time_1 = time.time()
model.fit(features_train, target_train)
time_2 = time.time()
prediction = model.predict(features_valid)
time_3 = time.time()
score = mean_squared_error(target_valid, prediction)
print('RMSE =', round(score**0.5), 'время на обучение =', round((time_2 - time_1), 1), 'времня на обучение =',  round((time_3 - time_2), 1))




,price,registrationyear,power,kilometer,registrationmonth,postalcode,vehicletype_convertible,vehicletype_coupe,vehicletype_other,vehicletype_other_1,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notrepaired_other_5,notrepaired_yes
0,480,1993,109,150000,0,70435,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,18300,2011,190,125000,5,66954,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,8,90480,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1500,2001,75,150000,6,91074,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,3600,2008,69,90000,7,60437,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,650,1995,102,150000,10,33775,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2200,2004,109,150000,8,67112,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,14500,2014,125,30000,8,94505,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,999,1998,101,150000,0,27472,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
10,2000,2004,105,150000,12,96224,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


RMSE = 2793 время на обучение = 28.4 времня на обучение = 0.2


### Кросс-валидация для оценки качества и скорости работы модели LinearRegression <a id="chapter2_3_1"></a>

In [9]:
features[numeric] = scaler.transform(features[numeric])
model = LinearRegression()
time_1 = time.time()
score = cross_val_score(model, features, target, cv = 4, scoring = make_scorer(rmse))
time_2 = time.time()
print('RMSE =', score.mean(), 'время на выполнение =', time_2 - time_1) 

RMSE = 2782.792173869746 время на выполнение = 112.6459174156189


# Анализ моделей <a id="chapter3"></a>

Для поиска оптимальных гиперпараметров моделей LGBMRegressor и CatBoostRegressor (в данном случае количество деревьев и глубина дерева) были использованы циклы со следующими парметрами: для количества деревьев - от 10 до 310 с шагом 50, для глубины дерева - от 2 до 10 с шагом 2. Анализируя полученные данные, можно сказать о том, что используя кросс-валидацию LGBMRegressor потратил времени на обучение и предсказание намного меньше, чем CatBoostRegressor (116 с против 2265 с). При этом метрика RMSE у LGBMRegressor получилась лучше: 1620 против 1730. Поэтому, можно сказать о том, что меняя только количество деревьев и глубину дерева, выигрывает LGBMRegressor. Что касается линейной регрессии, то в данной задаче она не смогла приблизиться к бустинговым моделям: RMSE = 2782, время на обучение и предсказывание = 140 с. 


In [14]:
final_result = pd.DataFrame({'RMSE' : ['1623', '1742 ', '2793 '],
                            'время обучения' : ['21.1', '530.4', '28.4'],
                            'время предсказания' : ['3.8', '0.8', '0.2']})
final_result.index = ['LGBMRegressor', 'CatBoostRegressor', 'LinearRegression']
display(final_result)

,RMSE,время обучения,время предсказания
LGBMRegressor,1623,21.1,3.8
CatBoostRegressor,1742,530.4,0.8
LinearRegression,2793,28.4,0.2


# Общий вывод <a id="chapter4"></a>

В данном проекте по предсказанию стоимости автомболей была выполнена предобработка данных, в ходе которой было вывлено много несоответствий. Данные в таком виде не годятся для моделей машинного обучения, т.к. есть очень много нелогичных записей, которые говорят о том, что один и тот же автомобиль может стоить на рынке 100 и 1000 евро. Чтобы модели начали работать лучше необходимо дополнить базу данных информацией (например, фотографиями а/м, количество собственников и т.д.). Несмотря на это, были обучены несколько моделей: LGBMRegressor, CatBoostRegressor, LinearRegression. Лучше всех себя показала модель LGBMRegressor.